In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.svm import SVR
import urllib.request
import tool_box as tb

In [2]:
df = pd.read_csv("./data/train.csv", index_col = 'laptop_ID')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           912 non-null    object 
 1   Product           912 non-null    object 
 2   TypeName          912 non-null    object 
 3   Inches            912 non-null    float64
 4   ScreenResolution  912 non-null    object 
 5   Cpu               912 non-null    object 
 6   Ram               912 non-null    object 
 7   Memory            912 non-null    object 
 8   Gpu               912 non-null    object 
 9   OpSys             912 non-null    object 
 10  Weight            912 non-null    object 
 11  Price_in_euros    912 non-null    float64
dtypes: float64(2), object(10)
memory usage: 92.6+ KB


In [4]:
df.OpSys.value_counts()

OpSys
Windows 10      741
Linux            48
No OS            44
Windows 7        29
Chrome OS        24
macOS            11
Windows 10 S      7
Mac OS X          6
Android           2
Name: count, dtype: int64

In [5]:
df.Company.value_counts()

Company
Lenovo       202
Dell         197
HP           194
Asus         121
Acer          74
MSI           37
Toshiba       34
Apple         17
Razer          6
Mediacom       6
Samsung        5
Microsoft      5
Xiaomi         3
Huawei         2
Chuwi          2
Google         2
Vero           2
Fujitsu        2
LG             1
Name: count, dtype: int64

In [6]:
df.sample(20)

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
laptop_ID,,,,,,,,,,,,
401,Dell,Inspiron 5567,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,AMD Radeon R7 M445,Linux,2.33kg,778.87
119,HP,Probook 440,Notebook,14.0,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,512GB SSD,Intel UHD Graphics 620,Windows 10,1.63kg,1084.00
1283,Lenovo,IdeaPad Y700-15ISK,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,8GB,1TB HDD,Nvidia GeForce GTX 960M,Windows 10,2.6kg,899.00
63,Lenovo,IdeaPad 520S-14IKB,Notebook,14.0,IPS Panel Full HD 1920x1080,Intel Core i3 7130U 2.7GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.7kg,599.00
709,Lenovo,IdeaPad 320-15AST,Notebook,15.6,Full HD 1920x1080,AMD A9-Series 9420 2.9GHz,4GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,399.00
114,Dell,XPS 13,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7560U 2.4GHz,8GB,256GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.23kg,1379.00
1177,HP,Spectre Pro,2 in 1 Convertible,13.3,Touchscreen 2560x1440,Intel Core i7 6600U 2.6GHz,8GB,512GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,2041.00
1108,Dell,Inspiron 5370,Ultrabook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,4GB,256GB SSD,AMD Radeon 530,Windows 10,1.4kg,818.35
1268,Dell,Inspiron 3552,Notebook,15.6,1366x768,Intel Pentium Quad Core N3710 1.6GHz,4GB,500GB HDD,Intel HD Graphics,Linux,2.2kg,324.00


In [78]:
df.Gpu.value_counts()

Gpu
Intel HD Graphics 620       185
Intel HD Graphics 520       125
Intel UHD Graphics 620       52
Nvidia GeForce GTX 1050      48
Nvidia GeForce 940MX         31
                           ... 
AMD Radeon RX 540             1
Nvidia Quadro M2000M          1
Nvidia GeForce GTX 940M       1
AMD Radeon R5 520             1
Nvidia GeForce GTX 1070M      1
Name: count, Length: 93, dtype: int64

In [7]:
import tool_box as tb

In [8]:
tb.describe_df(df)

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
DATA_TYPE,object,object,object,float64,object,object,object,object,object,object,object,float64
MISSINGS (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UNIQUE_VALUES,19,480,6,17,36,107,9,37,93,9,165,603
CARDIN (%),2.08,52.63,0.66,1.86,3.95,11.73,0.99,4.06,10.2,0.99,18.09,66.12


In [4]:
train_set, test_set = train_test_split(df, test_size = 0.20, random_state = 42)

- Elimino las filas que cointienen en la columna OpSys el nombre Mac Os X, porque al hacer el split no se encuentra en ambos sets. Se ha intentado hacer varios splits estratificados pero me entorpece en otras columnas asique elimino 6 instancias del dataset con la variable Mac Os X

In [5]:
train_set = train_set[train_set.Company != 'Huawey']

In [6]:
test_set = test_set[test_set.Company != 'Huawey']

In [7]:
train_set = train_set[train_set.OpSys != 'Android']

In [8]:
test_set = test_set[test_set.OpSys != 'Android']

In [9]:
target = 'Price_in_euros'

- Ram y Weight

In [10]:
train_set['ram_GB']=train_set.Ram.str.replace("GB", "").astype(int)
train_set['weight_kg']=train_set.Weight.str.replace("kg", "").astype(float)
test_set['ram_GB']=test_set.Ram.str.replace("GB", "").astype(int)
test_set['weight_kg']=test_set.Weight.str.replace("kg", "").astype(float)

In [16]:
tb.describe_df(train_set)

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros,ram_GB,weight_kg
DATA_TYPE,object,object,object,float64,object,object,object,object,object,object,object,float64,int64,float64
MISSINGS (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UNIQUE_VALUES,17,407,6,14,32,92,8,33,84,8,147,504,8,141
CARDIN (%),2.34,55.91,0.82,1.92,4.4,12.64,1.1,4.53,11.54,1.1,20.19,69.23,1.1,19.37


- Gpu

In [82]:
a = 'Intel HD Graphics 620'
b = a.split(' ')[0]
b

'Intel'

In [11]:
lista = []
for x in train_set['Gpu']:
    a = x.split(" ")[0]
    lista.append(a)

In [12]:
train_set['Gpu_new'] = lista
train_set.Gpu_new.value_counts()

Gpu_new
Intel     416
Nvidia    220
AMD        92
Name: count, dtype: int64

In [13]:
lista_test = []
for x in test_set['Gpu']:
    a = x.split(" ")[0]
    lista_test.append(a)

In [14]:
test_set['Gpu_new'] = lista_test
test_set.Gpu_new.value_counts()

Gpu_new
Intel     91
Nvidia    64
AMD       27
Name: count, dtype: int64

- Pixel

In [15]:
lista = []
for x in train_set['ScreenResolution']:
    a = x.split(" ")
    b = a[-1].split('x')
    c = int(b[0])*int(b[1])
    lista.append(c)

In [16]:
train_set['pixel'] = lista



In [17]:
lista_test = []
for x in test_set['ScreenResolution']:
    a = x.split(" ")
    b = a[-1].split('x')
    c = int(b[0])*int(b[1])
    lista_test.append(c)

test_set['pixel'] = lista_test

- Company

In [19]:
a = df.Company.unique()
lista_company = []
for x in a:
    lista_company.append(x)

In [20]:
otros = []
for company in lista_company:
    if df.loc[df['Company'] == company, 'Company'].value_counts()[0] < 7:
        otros.append(company)

C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\1552707568.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.loc[df['Company'] == company, 'Company'].value_counts()[0] < 7:
C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\1552707568.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.loc[df['Company'] == company, 'Company'].value_counts()[0] < 7:
C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\1552707568.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access 

In [21]:
for x in otros:
    if x in train_set['Company'].unique():
        train_set.loc[train_set['Company'] == x, 'Company'] = 'Otros'

In [22]:
for x in otros:
    if x in test_set['Company'].unique():
        test_set.loc[test_set['Company'] == x, 'Company'] = 'Otros'

In [23]:
train_set.Company.value_counts()

Company
Lenovo     161
Dell       158
HP         155
Asus       101
Acer        55
Toshiba     31
MSI         30
Otros       24
Apple       13
Name: count, dtype: int64

In [24]:
test_set.Company.value_counts()

Company
HP         39
Lenovo     39
Dell       39
Asus       20
Acer       19
Otros      12
MSI         7
Apple       4
Toshiba     3
Name: count, dtype: int64

- Cpu

In [28]:
lista_train = []
for x in train_set['Cpu']:
    a = tuple(x.split(" ")[0:3])
    union = " ".join(a)
    lista_train.append(union)

lista_train

['Intel Core i7',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core i5',
 'AMD A9-Series 9420',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i3',
 'AMD A9-Series 9420',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i5',
 'AMD A10-Series 9600P',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Celeron Dual',
 'Intel Core i5',
 'AMD A8-Series 7410',
 'Intel Core i5',
 'Intel Core i5',
 'AMD A12-Series 9720P',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i3',
 'Intel Core i5',
 'AMD E-Series 7110',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core M',
 'Intel Core i5',
 'Intel Celeron Dual',
 'Intel Core i7',
 'In

In [29]:
train_set['Cpu_new'] = lista_train
train_set.Cpu_new.value_counts()

Cpu_new
Intel Core i7               299
Intel Core i5               232
Intel Core i3                74
Intel Celeron Dual           48
Intel Pentium Quad           20
Intel Core M                 11
AMD A12-Series 9720P          5
AMD A6-Series 9220            5
Intel Atom x5-Z8350           4
Intel Celeron Quad            4
AMD A9-Series 9420            4
Intel Pentium Dual            3
AMD E-Series E2-9000e         2
AMD Ryzen 1700                2
AMD A8-Series 7410            2
AMD A10-Series 9600P          1
AMD E-Series E2-6110          1
AMD E-Series 7110             1
AMD A10-Series A10-9620P      1
AMD A6-Series A6-9220         1
AMD FX 9830P                  1
AMD E-Series 9000e            1
AMD A9-Series A9-9420         1
AMD A6-Series 7310            1
Intel Atom X5-Z8350           1
AMD A9-Series 9410            1
Intel Xeon E3-1535M           1
AMD E-Series 6110             1
Name: count, dtype: int64

In [30]:
a = train_set.Cpu_new.unique()
lista_company = []
for x in a:
    lista_company.append(x)

In [31]:
otros = []
for company in lista_company:
    if train_set.loc[train_set['Cpu_new'] == company, 'Cpu_new'].value_counts()[0] < 10:
        otros.append(company)

C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\3572950788.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if train_set.loc[train_set['Cpu_new'] == company, 'Cpu_new'].value_counts()[0] < 10:
C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\3572950788.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if train_set.loc[train_set['Cpu_new'] == company, 'Cpu_new'].value_counts()[0] < 10:
C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\3572950788.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with D

In [32]:
for x in otros:
    if x in train_set['Cpu_new'].unique():
        train_set.loc[train_set['Cpu_new'] == x, 'Cpu_new'] = 'Otros'

In [33]:
lista_test = []
for x in test_set['Cpu']:
    a = tuple(x.split(" ")[0:3])
    union = " ".join(a)
    lista_test.append(union)

lista_test

['Intel Core i7',
 'Intel Core i5',
 'Intel Core i5',
 'Intel Core i3',
 'Intel Celeron Dual',
 'Intel Celeron Dual',
 'Intel Core i5',
 'Intel Celeron Dual',
 'Intel Core i7',
 'Intel Core i5',
 'AMD A6-Series A6-9220',
 'Intel Core i5',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i5',
 'AMD Ryzen 1600',
 'Intel Celeron Dual',
 'Intel Core i7',
 'Intel Celeron Dual',
 'Intel Core i3',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i3',
 'Intel Core i3',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Celeron Dual',
 'Intel Core i7',
 'Intel Core i3',
 'Intel Atom x5-Z8300',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i3',
 'Intel Core i7',
 'Intel Core i5',
 'Intel Core i5',
 'AMD A10-Series A10-9620P',
 'Intel Celeron Dual',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core i7',
 'Intel Core i5',
 'I

In [34]:
test_set['Cpu_new'] = lista_test
test_set.Cpu_new.value_counts()

Cpu_new
Intel Core i7               75
Intel Core i5               50
Intel Core i3               19
Intel Celeron Dual          17
Intel Core M                 4
AMD A9-Series 9420           3
AMD A6-Series A6-9220        1
AMD Ryzen 1600               1
AMD A10-Series A10-9620P     1
Intel Atom x5-Z8300          1
AMD A9-Series A9-9420        1
Intel Atom x5-Z8350          1
AMD A10-Series 9620P         1
AMD A9-Series 9410           1
AMD A6-Series 9220           1
AMD E-Series E2-9000         1
Intel Xeon E3-1505M          1
Intel Pentium Quad           1
Intel Atom Z8350             1
Intel Celeron Quad           1
Name: count, dtype: int64

In [35]:
a = test_set.Cpu_new.unique()
lista_company = []
for x in a:
    lista_company.append(x)

In [36]:
otros = []
for company in lista_company:
    if company != 'Intel Pentium Quad':
        if test_set.loc[test_set['Cpu_new'] == company, 'Cpu_new'].value_counts()[0] < 4:
            otros.append(company)

C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\3002521287.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if test_set.loc[test_set['Cpu_new'] == company, 'Cpu_new'].value_counts()[0] < 4:
C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\3002521287.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if test_set.loc[test_set['Cpu_new'] == company, 'Cpu_new'].value_counts()[0] < 4:
C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\3002521287.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFra

In [38]:
for x in otros:
    if x in test_set['Cpu_new'].unique():
        test_set.loc[test_set['Cpu_new'] == x, 'Cpu_new'] = 'Otros'

In [39]:
test_set.Cpu_new.value_counts()

Cpu_new
Intel Core i7         75
Intel Core i5         50
Intel Core i3         19
Intel Celeron Dual    17
Otros                 16
Intel Core M           4
Intel Pentium Quad     1
Name: count, dtype: int64

 - OneHot Encoding  [Gpu_new, Company, TypeName, OpSys, Cpu_new]

In [40]:
train_set = pd.get_dummies(train_set, columns = ['Gpu_new'], dtype = int)
test_set= pd.get_dummies(test_set, columns = ['Gpu_new'], dtype = int)

In [41]:
train_set = pd.get_dummies(train_set, columns = ['TypeName'], dtype = int)
test_set= pd.get_dummies(test_set, columns = ['TypeName'], dtype = int)

In [42]:
train_set = pd.get_dummies(train_set, columns = ['Company'], dtype = int)
test_set= pd.get_dummies(test_set, columns = ['Company'], dtype = int)

In [43]:
train_set = pd.get_dummies(train_set, columns = ['Cpu_new'], dtype = int)
test_set= pd.get_dummies(test_set, columns = ['Cpu_new'], dtype = int)

In [44]:
train_set = pd.get_dummies(train_set, columns = ['OpSys'], dtype = int)
test_set= pd.get_dummies(test_set, columns = ['OpSys'], dtype = int)

In [68]:
target = 'Price_in_euros'

In [ ]:
feat_num = train_set.describe().columns.to_list()
feat_num

In [48]:
feat_num.remove(target)
feat_num.remove('OpSys_Mac OS X')

In [54]:
feat_test = test_set.describe().columns.to_list()
feat_test.remove(target)
print(len(feat_test))
print(len(feat_num))

36
36


- Selección de features

ANOVA para las columnas categoricas One Hot Encoding

In [64]:
from sklearn.feature_selection import SelectKBest, f_classif

X = train_set[feat_num[4:]]
Y = train_set[target]
selector = SelectKBest(f_classif, k=25)
x_data_kbest = selector.fit_transform(X, Y)
X_train_kbest = pd.DataFrame(x_data_kbest, columns = selector.get_feature_names_out())
lista_ANOVA = X_train_kbest.columns.to_list()


c:\Users\MARTA\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


Pearson para las columnas numéricas

In [69]:
feat_num.append('Price_in_euros')
corr= train_set[feat_num[:4] + [target]].corr()
serie_corr = np.abs(corr[target])
serie_corr.sort_values(ascending = False)

Price_in_euros    1.000000
ram_GB            0.744023
pixel             0.497508
weight_kg         0.200713
Inches            0.047853
Name: Price_in_euros, dtype: float64

In [70]:
lista_ANOVA.append('ram_GB')
lista_ANOVA.append('pixel')
lista_ANOVA.append('weight_kg')

In [71]:
feat_num.remove(target)

Seleccion por eliminación recursiva de features

In [73]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

X_train_recur = train_set[feat_num].copy()
y_train_recur = train_set[target]

rf_RFE = RandomForestRegressor(random_state= 42) 

rfe = RFE(estimator = rf_RFE,
          n_features_to_select= 20, # Iterará hasta quedarse con 5
          step = 1) # Elimina de una en una, pero es configurable

rfe.fit(X_train_recur, y_train_recur)
print(rfe.ranking_)
X_train_rfe = pd.DataFrame(rfe.transform(X_train_recur), columns = rfe.get_feature_names_out())
lista_RFE = X_train_rfe.columns.to_list()

[ 1  1  1  1  1  6  1  1  2 12  1  1  1 10 13  1  1  1  1  3  1  5  8  7
  1  1  1 14  4 16 11  9  1 17  1 15]


Seleccion secuencial de Features

In [76]:
from sklearn.feature_selection import SequentialFeatureSelector

rf_SFS = RandomForestRegressor(random_state = 42)

sfs_forward = SequentialFeatureSelector(rf_SFS,
                                        n_features_to_select = 20,
                                        cv = 4,
                                        scoring = "neg_mean_squared_error")


sfs_forward.fit(X_train_recur, y_train_recur)

X_train_sfs = pd.DataFrame(sfs_forward.transform(X_train_recur), columns = sfs_forward.get_feature_names_out())
lista_SFS = X_train_sfs.columns.to_list()
print("Elegidasd SFS:", sfs_forward.get_feature_names_out())


Elegidasd SFS: ['Inches' 'ram_GB' 'weight_kg' 'pixel' 'TypeName_2 in 1 Convertible'
 'TypeName_Notebook' 'TypeName_Ultrabook' 'TypeName_Workstation'
 'Company_Acer' 'Company_Apple' 'Company_Asus' 'Company_Lenovo'
 'Company_Otros' 'Cpu_new_Intel Core i3' 'Cpu_new_Intel Core i5'
 'Cpu_new_Intel Core i7' 'Cpu_new_Intel Pentium Quad' 'OpSys_Chrome OS'
 'OpSys_No OS' 'OpSys_Windows 10 S']


In [ ]:
listas = {}
listas['Todas'] = feat_num
listas['ANOVA_corr'] = lista_ANOVA
listas['RFE'] = lista_RFE
listas['SFS'] = lista_SFS

In [90]:
from sklearn.model_selection import cross_val_score
rf_reg = RandomForestRegressor( random_state= 42)

puntuaciones = {}
for x,y in listas.items():
    score = np.mean(cross_val_score(rf_reg, train_set[y], train_set[target], cv = 5, scoring='neg_mean_squared_error'))
    puntuaciones[x] = score

puntuaciones

{'Todas': np.float64(-97043.74433245696),
 'ANOVA_corr': np.float64(-99241.38027172245),
 'RFE': np.float64(-98993.89322913598),
 'SFS': np.float64(-96664.55799566733)}

In [91]:
for model, valores in puntuaciones.items():
    print(f"Model <{model}>, RMSE_CV: {np.sqrt(-np.mean(valores))}")


Model <Todas>, RMSE_CV: 311.51844942548263
Model <ANOVA_corr>, RMSE_CV: 315.0259993583426
Model <RFE>, RMSE_CV: 314.63295000545634
Model <SFS>, RMSE_CV: 310.9092439855517


In [88]:
from catboost import CatBoostRegressor
cat_reg = CatBoostRegressor(random_state= 42, verbose = False)

puntuaciones_cat = {}
for x,y in listas.items():
    score = np.mean(cross_val_score(cat_reg, train_set[y], train_set[target], cv = 5, scoring='neg_mean_squared_error'))
    puntuaciones_cat[x] = score

puntuaciones_cat

{'Todas': np.float64(-86454.7272262659),
 'ANOVA_corr': np.float64(-92041.57017824326),
 'RFE': np.float64(-86343.65896306661),
 'SFS': np.float64(-85891.47978657702)}

In [89]:
for model, valores in puntuaciones_cat.items():
    print(f"Model <{model}>, RMSE_CV: {np.sqrt(-np.mean(valores))}")

Model <Todas>, RMSE_CV: 294.03184729934594
Model <ANOVA_corr>, RMSE_CV: 303.3835364324229
Model <RFE>, RMSE_CV: 293.8429154549529
Model <SFS>, RMSE_CV: 293.0724821380831


In [92]:
from lightgbm import LGBMRegressor
lgb_reg = LGBMRegressor(random_state= 42, verbose = -100)

puntuaciones_lgb = {}
for x,y in listas.items():
    score = np.mean(cross_val_score(lgb_reg, train_set[y], train_set[target], cv = 5, scoring='neg_mean_squared_error'))
    puntuaciones_lgb[x] = score

for model, valores in puntuaciones_lgb.items():
    print(f"Model <{model}>, RMSE_CV: {np.sqrt(-np.mean(valores))}")

Model <Todas>, RMSE_CV: 324.44907788920085
Model <ANOVA_corr>, RMSE_CV: 337.4710109847912
Model <RFE>, RMSE_CV: 328.2216094534935
Model <SFS>, RMSE_CV: 329.4154098147236


In [93]:
X_train = train_set[feat_num]
y_train = train_set[target]
X_test = test_set[feat_num]
y_test = test_set[target]


In [94]:
X_train_SFS = train_set[lista_SFS]
y_train = train_set[target]
X_test_SFS = test_set[lista_SFS]
y_test = test_set[target]


In [95]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [96]:


param_grid= {'depth': [3, 6, 12],
          'learning_rate': [0.1, 0.2, 0.3, 0.4],
          #'bagging_fraction': [0.3,0.6,1], No hay hiperparámetro equivalente
          'colsample_bylevel': [0.5,1],
          'iterations': [100, 250, 500, 750],
          "border_count": [125,250]
          }


cat_ran = RandomizedSearchCV(cat_reg, 
                               cv = 3,
                               n_iter = 3,
                               param_distributions= param_grid, 
                               scoring = "neg_mean_squared_error")


cat_ran.fit(X_train, y_train)
print("CatBoost best_score:", np.sqrt(-cat_ran.best_score_))


CatBoost best_score: 298.9496293116841


In [97]:
y_pred_random = cat_ran.best_estimator_.predict(X_test)
print("MAE:", mean_absolute_error(y_test,y_pred_random))
print("MAPE:", mean_absolute_percentage_error(y_test,y_pred_random))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_random)))

MAE: 211.07266428106894
MAPE: 0.18431063781822069
RMSE: 329.0850619759376


In [98]:
param_grid= {'depth': [3, 6, 12],
          'learning_rate': [0.1, 0.2, 0.3, 0.4],
          #'bagging_fraction': [0.3,0.6,1], No hay hiperparámetro equivalente
          'colsample_bylevel': [0.5,1],
          'iterations': [100, 250, 500, 750],
          "border_count": [125,250]
          }
cat_grid = GridSearchCV(cat_reg, 
                            cv = 3,
                            param_grid= param_grid, 
                            scoring = "neg_mean_squared_error",
                            n_jobs=-1)


cat_grid.fit(X_train, y_train)
print("Catboost best_score:", np.sqrt(-cat_grid.best_score_))

Catboost best_score: 292.6827174394581


In [99]:
cat_grid.best_params_

{'border_count': 125,
 'colsample_bylevel': 0.5,
 'depth': 3,
 'iterations': 750,
 'learning_rate': 0.1}

In [100]:
y_pred = cat_grid.best_estimator_.predict(X_test)
print("MAE:", mean_absolute_error(y_test,y_pred))
print("MAPE:", mean_absolute_percentage_error(y_test,y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

MAE: 211.3835329272711
MAPE: 0.18814625607040358
RMSE: 324.58933488875357


In [101]:
param_grid= {'depth': [3, 6, 12],
          'learning_rate': [0.1, 0.2, 0.3, 0.4],
          #'bagging_fraction': [0.3,0.6,1], No hay hiperparámetro equivalente
          'colsample_bylevel': [0.5,1],
          'iterations': [100, 250, 500, 750],
          "border_count": [125,250]
          }
cat_grid_SFS = GridSearchCV(cat_reg, 
                            cv = 3,
                            param_grid= param_grid, 
                            scoring = "neg_mean_squared_error",
                            n_jobs=-1)


cat_grid_SFS.fit(X_train_SFS, y_train)
print("Catboost best_score:", np.sqrt(-cat_grid_SFS.best_score_))

Catboost best_score: 300.958952695309


In [102]:
y_pred = cat_grid_SFS.best_estimator_.predict(X_test_SFS)
print("MAE:", mean_absolute_error(y_test,y_pred))
print("MAPE:", mean_absolute_percentage_error(y_test,y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

MAE: 205.59404227519408
MAPE: 0.19154078455875254
RMSE: 309.9487673579557


## Preparamos el X_pred 

In [103]:
X_pred = pd.read_csv("./data/test.csv", index_col = 'laptop_ID')
X_pred.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
laptop_ID,,,,,,,,,,,
209,Lenovo,Legion Y520-15IKBN,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1060,No OS,2.4kg
1281,Acer,Aspire ES1-531,Notebook,15.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,500GB HDD,Intel HD Graphics 400,Linux,2.4kg
1168,Lenovo,V110-15ISK (i3-6006U/4GB/1TB/No,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,Intel HD Graphics 520,No OS,1.9kg
1231,Dell,Inspiron 7579,2 in 1 Convertible,15.6,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,2.191kg
1020,HP,ProBook 640,Notebook,14.0,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.95kg


- Tratamiento columnas Ram y Weight

In [104]:
X_pred['ram_GB']=X_pred.Ram.str.replace("GB", "").astype(int)
X_pred['weight_kg']=X_pred.Weight.str.replace("kg", "").astype(float)

- Tratamiento columna ScreenResolution

In [105]:
lista = []
for x in X_pred['ScreenResolution']:
    a = x.split(" ")
    b = a[-1].split('x')
    c = int(b[0])*int(b[1])
    lista.append(c)

X_pred['pixel'] = lista

- Tratamiento columna Company

In [106]:
X_pred.loc[X_pred['Company'] == 'Samsung', 'Company'] = 'Otros'

In [107]:
a = X_pred.Company.unique()
lista_company_pred = []
for x in a:
    lista_company_pred.append(x)

In [108]:
X_pred.Company.value_counts()

Company
Dell         100
Lenovo        95
HP            80
Asus          37
Acer          29
MSI           17
Toshiba       14
Otros          4
Apple          4
Vero           2
LG             2
Mediacom       1
Razer          1
Fujitsu        1
Google         1
Xiaomi         1
Chuwi          1
Microsoft      1
Name: count, dtype: int64

In [109]:
otros = []
for company in lista_company_pred:
    if X_pred.loc[X_pred['Company'] == company, 'Company'].value_counts()[0] < 4:
        otros.append(company)

C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\585093429.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if X_pred.loc[X_pred['Company'] == company, 'Company'].value_counts()[0] < 4:
C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\585093429.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if X_pred.loc[X_pred['Company'] == company, 'Company'].value_counts()[0] < 4:
C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\585093429.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior

In [110]:
for x in otros:
    if x in X_pred['Company'].unique():
        X_pred.loc[X_pred['Company'] == x, 'Company'] = 'Otros'

In [111]:
X_pred.Company.value_counts()

Company
Dell       100
Lenovo      95
HP          80
Asus        37
Acer        29
MSI         17
Otros       15
Toshiba     14
Apple        4
Name: count, dtype: int64

- Tratamiento GPU

In [112]:
lista = []
for x in X_pred['Gpu']:
    a = x.split(" ")[0]
    lista.append(a)

X_pred['Gpu_new'] = lista
X_pred.Gpu_new.value_counts()

Gpu_new
Intel     213
Nvidia    116
AMD        61
ARM         1
Name: count, dtype: int64

In [113]:
X_pred.loc[X_pred['Gpu_new'] == 'ARM', 'Gpu_new'] = 'Nvidia'
X_pred.Gpu_new.value_counts()

Gpu_new
Intel     213
Nvidia    117
AMD        61
Name: count, dtype: int64

In [67]:
X_pred.TypeName.value_counts()

TypeName
Notebook              218
Gaming                 62
Ultrabook              55
2 in 1 Convertible     41
Workstation             9
Netbook                 6
Name: count, dtype: int64

In [69]:
X_pred.OpSys.value_counts()

OpSys
Windows 10      331
No OS            22
Windows 7        16
Linux            14
Chrome OS         3
macOS             2
Mac OS X          2
Windows 10 S      1
Name: count, dtype: int64

In [114]:
X_pred.loc[X_pred['OpSys'] == 'Mac OS X', 'OpSys'] = 'macOS'
X_pred.OpSys.value_counts()

OpSys
Windows 10      331
No OS            22
Windows 7        16
Linux            14
macOS             4
Chrome OS         3
Windows 10 S      1
Name: count, dtype: int64

- Tratamiento columna Cpu

In [117]:
lista = []
for x in X_pred['Cpu']:
    a = tuple(x.split(" ")[0:3])
    union = " ".join(a)
    lista.append(union)

X_pred['Cpu_new'] = lista
X_pred.Cpu_new.value_counts()


Cpu_new
Intel Core i7             153
Intel Core i5             141
Intel Core i3              43
Intel Celeron Dual         15
Intel Pentium Quad          6
AMD A9-Series 9420          5
Intel Core M                4
Intel Celeron Quad          3
AMD A8-Series 7410          2
AMD A6-Series 9220          2
Intel Atom x5-Z8550         2
AMD A12-Series 9720P        2
Intel Atom X5-Z8350         1
AMD FX 8800P                1
Samsung Cortex A72&A53      1
AMD A10-Series 9620P        1
AMD A9-Series 9410          1
Intel Xeon E3-1505M         1
AMD A10-Series 9600P        1
AMD Ryzen 1700              1
AMD A12-Series 9700P        1
AMD E-Series 9000           1
AMD E-Series 7110           1
Intel Xeon E3-1535M         1
AMD A4-Series 7210          1
Name: count, dtype: int64

In [118]:
a = X_pred.Cpu_new.unique()
lista_company = []
for x in a:
    lista_company.append(x)

In [120]:
otros = []
for company in lista_company:
    if company != 'Intel Core M':
        if X_pred.loc[X_pred['Cpu_new'] == company, 'Cpu_new'].value_counts()[0] < 6:
            otros.append(company)

C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\365776222.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if X_pred.loc[X_pred['Cpu_new'] == company, 'Cpu_new'].value_counts()[0] < 6:
C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\365776222.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if X_pred.loc[X_pred['Cpu_new'] == company, 'Cpu_new'].value_counts()[0] < 6:
C:\Users\MARTA\AppData\Local\Temp\ipykernel_15896\365776222.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior

In [121]:
for x in otros:
    if x in X_pred['Cpu_new'].unique():
        X_pred.loc[X_pred['Cpu_new'] == x, 'Cpu_new'] = 'Otros'

In [122]:
X_pred.Cpu_new.value_counts()

Cpu_new
Intel Core i7         153
Intel Core i5         141
Intel Core i3          43
Otros                  29
Intel Celeron Dual     15
Intel Pentium Quad      6
Intel Core M            4
Name: count, dtype: int64

- OneHot Encoding para las columnas Company, TypeName y OpSys

In [123]:
X_pred = pd.get_dummies(X_pred, columns = ['Company'], dtype = int)
X_pred = pd.get_dummies(X_pred, columns = ['TypeName'], dtype = int)
X_pred = pd.get_dummies(X_pred, columns = ['OpSys'], dtype = int)
X_pred = pd.get_dummies(X_pred, columns = ['Gpu_new'], dtype = int)
X_pred = pd.get_dummies(X_pred, columns = ['Cpu_new'], dtype = int)

- Seleccion de features

In [124]:
feat_num = X_pred.describe().columns.to_list()
feat_num

['Inches',
 'ram_GB',
 'weight_kg',
 'pixel',
 'Company_Acer',
 'Company_Apple',
 'Company_Asus',
 'Company_Dell',
 'Company_HP',
 'Company_Lenovo',
 'Company_MSI',
 'Company_Otros',
 'Company_Toshiba',
 'TypeName_2 in 1 Convertible',
 'TypeName_Gaming',
 'TypeName_Netbook',
 'TypeName_Notebook',
 'TypeName_Ultrabook',
 'TypeName_Workstation',
 'OpSys_Chrome OS',
 'OpSys_Linux',
 'OpSys_No OS',
 'OpSys_Windows 10',
 'OpSys_Windows 10 S',
 'OpSys_Windows 7',
 'OpSys_macOS',
 'Gpu_new_AMD',
 'Gpu_new_Intel',
 'Gpu_new_Nvidia',
 'Cpu_new_Intel Celeron Dual',
 'Cpu_new_Intel Core M',
 'Cpu_new_Intel Core i3',
 'Cpu_new_Intel Core i5',
 'Cpu_new_Intel Core i7',
 'Cpu_new_Intel Pentium Quad',
 'Cpu_new_Otros']

In [125]:
len(feat_num)

36

In [126]:
predictions_submit = cat_grid.best_estimator_.predict(X_pred[feat_num])
predictions_submit

array([1376.66443498,  301.18682052,  346.75786575,  888.46475674,
        986.42049913,  335.90096675,  848.14862985,  847.10877205,
       1235.25346439,  398.91405917, 2335.22410597, 1360.72562514,
        597.74861334, 1500.3884615 ,  776.14782324,  763.36261166,
       2563.06194099, 1409.7329468 , 2093.22843824,  681.59706121,
       1337.12184013,  281.21990827,  853.62687948,  917.698297  ,
        461.86880507,  810.72604081,  690.46544269, 1279.3234132 ,
       2774.10471463, 1125.89701353, 2063.79911207,  445.4117184 ,
        843.83906054, 3110.29245694, 1947.65053367, 1935.51026389,
        671.66083303, 1404.5312111 ,  923.47301709, 1760.9941373 ,
        751.46180291,  820.94769318,  474.19668867, 1372.83309796,
       1543.23361966, 1052.27306564, 1056.32512814,  512.3981101 ,
        715.87721667,  392.7900047 , 1602.18729752,  868.16529627,
       1125.84951864,  499.4738101 , 2013.20298568, 1769.48608293,
        708.14563991, 1039.28691782,  973.12245951,  600.63412

In [127]:
submission = pd.DataFrame({'laptop_ID': X_pred.index, 'Price_in_euros': predictions_submit})

In [128]:
submission.head()

,laptop_ID,Price_in_euros
0,209,1376.664435
1,1281,301.186821
2,1168,346.757866
3,1231,888.464757
4,1020,986.420499


In [129]:
submission.shape

(391, 2)

In [130]:
sample = pd.read_csv("data/sample_submission.csv")

In [131]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                df_to_submit.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [132]:
chequeador(submission)

You're ready to submit!
